In [1]:
import YAML

In [2]:
struct Horn
    name::String
    id::Int
    polid::String
    moduleid::Int
    color::String
    orientation::Array{Float64, 1}
    fwhm_x_deg::Float64
    fwhm_y_deg::Float64
    main_spillover::Float64
    sub_spillover::Float64
    xpd_db::Float64
    directivity_dbi::Float64
    ellipticity::Float64
end

function parsefpdict(fpdict::Dict{Any, Any})
    focalplane = Dict{String, Horn}()
    for (key, value) in fpdict["horns"]
        refhorn = if haskey(fpdict["pairs"], key)
            refhornname = fpdict["pairs"][key]
            fpdict["horns"][refhornname]
        else
            value
        end
        
        focalplane[key] = Horn(key,
                               value["id"],
                               value["polarimeter_id"],
                               value["module_id"],
                               value["color"],
                               value["orientation"],
                               refhorn["fwhm_x_deg"],
                               refhorn["fwhm_y_deg"],
                               refhorn["main_spillover"],
                               refhorn["sub_spillover"],
                               refhorn["xpd_db"],
                               refhorn["directivity_dbi"],
                               refhorn["ellipticity"])
    end
    
    focalplane
end

parsefpdict (generic function with 1 method)

In [3]:
struct Detector
    id::Int
    name::String
    center_frequency_hz::Float64
    bandwidth_hz::Float64
    alpha_i::Float64
    alpha_q::Float64
    alpha_u::Float64
    fknee_q_hz::Float64
    fknee_u_hz::Float64
    wn_q_k2_hz::Float64
    wn_u_k2_hz::Float64
end

struct InstrumentDB
    focalplane::Dict{String, Horn}
    detectors::Dict{Int, Detector}
end

function parsedetdict(detdict::Dict{Any, Any})
    detectors = Dict{Int, Detector}()
    for curdet in detdict["polarimeters"]
        id = curdet["id"]
        detectors[id] = Detector(id,
                                 curdet["name"],
                                 get(curdet, "center_frequency_hz", 0.0),
                                 get(curdet, "bandwidth_hz", 0.0),
                                 get(curdet, "alpha_i", 0.0),
                                 get(curdet, "alpha_q", 0.0),
                                 get(curdet, "alpha_u", 0.0),
                                 get(curdet, "fknee_q_hz", 0.0),
                                 get(curdet, "fknee_u_hz", 0.0),
                                 get(curdet, "wn_q_k2_hz", 0.0),
                                 get(curdet, "wn_q_k2_hz", 0.0))
    end
    
    detectors
end

parsedetdict (generic function with 1 method)

In [4]:
function InstrumentDB(dbpath)
    focalplanedict = open(joinpath(dbpath, "strip_focal_plane.yaml")) do f
        YAML.load(f)
    end
    
    detectordict = open(joinpath(dbpath, "strip_detectors.yaml")) do f
        YAML.load(f)
    end
    
    InstrumentDB(parsefpdict(focalplanedict), parsedetdict(detectordict))
end

InstrumentDB

In [6]:
InstrumentDB(raw"..\stripeline\instrument")

InstrumentDB(Dict("I5"=>Horn("I5", 5, "STRIP00", 5, "indigo", [-0.0107767, 0.0187696, 0.999766], 0.347, 0.345, 0.017151, 0.013704, 43.93, 54.75, 1.006),"O1"=>Horn("O1", 15, "STRIP00", 1, "orange", [-0.0099489, -0.0861267, 0.996235], 0.347, 0.349, 0.017318, 0.013305, 42.17, 54.69, 1.006),"V3"=>Horn("V3", 31, "STRIP00", 3, "violet", [0.0809495, 0.0335947, 0.996152], 0.355, 0.343, 0.01646, 0.013881, 40.85, 54.66, 1.034),"G4"=>Horn("G4", 46, "STRIP00", 4, "green", [-0.0468401, 0.0524999, 0.997522], 0.348, 0.346, 0.017683, 0.013402, 41.45, 54.71, 1.004),"I4"=>Horn("I4", 4, "STRIP00", 4, "indigo", [0.010898, 0.0187675, 0.999764], 0.348, 0.345, 0.016905, 0.01397, 44.19, 54.74, 1.009),"O4"=>Horn("O4", 18, "STRIP00", 4, "orange", [0.0111748, -0.048671, 0.998752], 0.348, 0.346, 0.017042, 0.014065, 42.97, 54.72, 1.007),"R5"=>Horn("R5", 26, "STRIP00", 5, "red", [0.0477568, -0.0149849, 0.998747], 0.351, 0.344, 0.016735, 0.014259, 41.98, 54.71, 1.02),"V1"=>Horn("V1", 29, "STRIP00", 1, "violet", [0.0